In [1]:
import os
import pandas as pd
from google.cloud import bigquery as bq

# ref: https://drive.google.com/file/d/1IHYxjtUo5hQjj81CLBH6RNoQM05oojBF/view

""" Dependencies of note:
- google-cloud
- db-dtypes
"""

' Dependencies of note:\n- google-cloud\n- db-dtypes\n'

In [2]:
# CREDENTIAL
""" google cloud credentials--> https://console.cloud.google.com/projectselector2/apis/credentials?supportedpurview=project&authuser=3
- https://developers.google.com/workspace/guides/create-credentials#google-cloud-console
- gcloud account --> make a project --> service account --> api key for service account
Credential Loc:
https://console.cloud.google.com/iam-admin/serviceaccounts/details/113699365795205553034/keys?authuser=3&project=codegen-404518&supportedpurview=project
--> download (https://pypi.org/project/google-cloud-bigquery/)
"""

cwd = os.getcwd()
secret_dir = "secret/"
api_key = cwd + "/" + secret_dir + os.listdir(secret_dir)[0]
assert api_key[-5:] == ".json"  # confirm that it was found
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = api_key

In [3]:
# From Google BigQuery  (https://pypi.org/project/google-cloud-bigquery/)
client = bq.Client()

# use api key for the session
def run_query(query_string):
    dataframe = (
        client.query(query_string)
        .result()
        .to_dataframe(
            create_bqstorage_client=True,
        )
    )
    print(dataframe.head())
    return dataframe

## Prelim Tests

In [4]:
# query documentaiton: https://googleapis.dev/nodejs/bigquery/latest/BigQuery.html

# query_string="""
# SELECT f.repo_name, f.path, c.copies, c.size, c.content
#  FROM `bigquery-public-data.github_repos.files` AS f 
#  JOIN `bigquery-public-data.github_repos.contents` AS c 
#  ON f.id = c.id 
#  WHERE 
#  NOT c.binary 
#  AND ((f.path LIKE '%.cbl') 
#  AND (c.size BETWEEN 1
#  AND 500))
# """

query_string="""SELECT f.repo_name, c.content
FROM `bigquery-public-data.github_repos.files` AS f
JOIN `bigquery-public-data.github_repos.contents` AS c
ON f.id = c.id
WHERE
NOT c.binary
AND f.path LIKE '%.py'
AND REGEXP_CONTAINS(c.content, r'(?m)^\s*assert ')
LIMIT 5"""


# query = """
#     SELECT corpus AS title, COUNT(word) AS unique_words
#     FROM `bigquery-public-data.samples.shakespeare`
#     GROUP BY title
#     ORDER BY unique_words
#     DESC LIMIT 10
# """
# results = bqclient.query(query)

In [5]:
original_df = run_query(query_string)
original_df

               repo_name                                            content
0      Bjay1435/capstone  #  Author:      Fred L. Drake, Jr.\n#         ...
1  cython-testbed/pandas  """\nDefine the SeriesGroupBy, DataFrameGroupB...
2       alanwells/donkey  #!/usr/bin/env python3\n# -*- coding: utf-8 -*...
3           htzy/bigfour  """\nA ModuleStore that knows about a special ...
4       chriskmanx/qmole  """SCons.Tool.install\n\nTool-specific initial...


,repo_name,content
0,Bjay1435/capstone,"# Author: Fred L. Drake, Jr.\n# ..."
1,cython-testbed/pandas,"""""""\nDefine the SeriesGroupBy, DataFrameGroupB..."
2,alanwells/donkey,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...
3,htzy/bigfour,"""""""\nA ModuleStore that knows about a special ..."
4,chriskmanx/qmole,"""""""SCons.Tool.install\n\nTool-specific initial..."


In [6]:
df = original_df.copy(deep=True)

In [43]:
conditionals = dict([[cond, i] for i, cond in enumerate(["==", "!=", "<=", ">=", "<", ">"])])
compounding_statements = ["and", "or"]  # TODO: properly account for OR

def get_assertions(func, is_split=True):
    """
    Format: "assert [expression], [return_string]"
    """
    out = []
    lines = [temp.strip() for temp in func.split('\n') if "assert" in temp]
    # TODO: maybe save some context around the assertion?
    ind = 0
    while ind < len(lines):
        data = lines[ind].strip()
        # TODO: handle or & and statements!
        start = data.find('assert')
        if start != -1:
            # account for combination statements
            for statement in compounding_statements:
                add_statement = data.find(statement)
                if add_statement != -1:
                    extra_line = data[add_statement+len(statement):]
                    lines.insert(ind+1, "assert "+extra_line)
                    data = data[:add_statement].strip()
            
            com = data.find(',')   # parsing out return_string
            if com != -1:
                data = data[:com]

            if is_split:
                data = [var.strip() for var in data.split()]
                assert data[0] == "assert", "something was found before the assertion in this line"
                data = data[1:]
                
                condition = True  # assertion [variable] == condition by default
                if data[0] == "not":  # accounting for not
                    condition = False
                    data = data[1:]
                    
                assert len(data) >= 1, "empty assertion found?: " + data
                if len(data) == 1:  # adding == to simlify
                    data = data + ["==", str(condition)]
                
                for i in range(len(data)):
                    if data[i] == "is":  # simplifying is to ==
                        data[i] = "=="
                    if data[i] in conditionals.keys():  # com
                        data = [' '.join(data[:i]), data[i], ' '.join(data[i+1:])]  # conditionals[data[i]]
                        break
            assert len(data) == 3, "found conditional-less assertion:\n" + data
            out.append(data)
        ind += 1
    return out

df["assertions"] = df["content"].apply(lambda code: get_assertions(code))
df

,repo_name,content,variables,assertions
0,Bjay1435/capstone,"# Author: Fred L. Drake, Jr.\n# ...",[Pretty-print a Python object to a stream [def...,"[[indent, >=, 0], [depth, ==, None], [depth, >..."
1,cython-testbed/pandas,"""""""\nDefine the SeriesGroupBy, DataFrameGroupB...","[self, slice_axis, ...","[[new_axes[0].equals(items), ==, True], [args,..."
2,alanwells/donkey,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...,"[self, path, inputs = None, types = None, ...","[[len(self., ==, True], [ig['inputs']), ==, le..."
3,htzy/bigfour,"""""""\nA ModuleStore that knows about a special ...","[log, item, item.location, self, usage_key...","[[item.get('_id').get('revision'), !=, MongoRe..."
4,chriskmanx/qmole,"""""""SCons.Tool.install\n\nTool-specific initial...","[__revision__, _INSTALLED_FILES, _UNIQUE_INSTA...","[[len(target)==len(source), ==, True]]"


In [87]:
banned_vars = ['', '*', 'self']
def get_variables(func, verbose=False):
    out = []
    for line in func.split('\n'):
        line = line.strip()
        if "def " in line:  # add params if its a function
            start = line.find('(')
            end = line.find(')')
            for new_param in line[start+1:end].split(','):
                default = new_param.find("=")
                if default != -1:
                    new_param = new_param[:default]
                new_param = new_param.strip()
                if new_param not in out and new_param not in banned_vars:
                    if verbose:
                        print("*Found  {", new_param, "}  at:\n", line, '\n')
                    out.append(new_param)
        else: # add variables if equals operation
            find_var = line.find(' = ')
            if find_var != -1:
                new_var = line[:find_var].strip()
                
                if ',' in new_var: # handle tuple equalities edge case (ex: a, b, c = fn_output())
                    var_list = [tuple_var.strip() for tuple_var in new_var.split(',')]
                else:
                    var_list = [new_var]
                for new_var in var_list:
                    if new_var not in out and new_var not in banned_vars:
                        if verbose:
                            print("**Found  {", new_var, "}  at:\n", line, '\n')
                        out.append(new_var)
            # TODO: handle indexing
    return out

# out = get_variables(df.sample()["content"].iloc[0])
get_vars = lambda code: get_variables(code)
df["variables"] = df["content"].apply(get_vars)
df

,repo_name,content,variables,assertions
0,Bjay1435/capstone,"# Author: Fred L. Drake, Jr.\n# ...","[__all__, object, stream, indent, width, depth...","[[indent, >=, 0], [depth, ==, None], [depth, >..."
1,cython-testbed/pandas,"""""""\nDefine the SeriesGroupBy, DataFrameGroupB...","[slice_axis, slicer, how, alt, numeric_only, n...","[[new_axes[0].equals(items), ==, True], [args,..."
2,alanwells/donkey,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...,"[path, inputs, types, self.path, self.current_...","[[len(self., ==, True], [ig['inputs']), ==, le..."
3,htzy/bigfour,"""""""\nA ModuleStore that knows about a special ...","[log, item, item.location, usage_key, depth, r...","[[item.get('_id').get('revision'), !=, MongoRe..."
4,chriskmanx/qmole,"""""""SCons.Tool.install\n\nTool-specific initial...","[__revision__, _INSTALLED_FILES, _UNIQUE_INSTA...","[[len(target)==len(source), ==, True]]"


## TODO: figure out filtering constraints that make for good assertions
- Goodness Criteria: (what is a good reference to optimize LLMs with
    - addative --> look at some edge case
    - comprehensive --> > 1 assertion in code block
    - fits our schema
        - assert [A,B,C, (int)] [==, >=, <=, !=] [A,B,C, (int)]
    - 

## Observations:
    - columns = 'repo_name', 'ref', 'path', 'mode', 'id', 'symlink_target', 'id_1', 'size', 'content', 'binary', 'copies'
    - variable data for comparison
        - len()
        - element index
        - boolean function
        
        
        
### Ideas: (quality data)
    - order results by quantity of asserted lines to lines of code in the repository
        - order results by optimal variable complexity? 2 < #vars < 6
        - split results by 
    - weight asserted lines by how neatly they fit our schedma (assert a [cond] b)
    - 